<a href="https://colab.research.google.com/github/ozakiryota/cart_pole/blob/main/cart_pole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cart-Pole


Install OpenAI Gym (https://gym.openai.com)

In [ ]:
!pip install gym

Install the packages for visualizing Gym

In [ ]:
!apt update
!apt install xvfb
!pip install pyvirtualdisplay

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downlo

## Random move

Import Gym

In [ ]:
import gym

Import the packages for visualizing Gym

In [ ]:
import base64
import io
from gym.wrappers import Monitor
from IPython import display
from pyvirtualdisplay import Display

Move the cart-pole randomly

In [ ]:
d = Display()
d.start()

env = Monitor(gym.make('CartPole-v0'),'./videos/random_move/', force=True)
print("env.observation_space.shape = ", env.observation_space.shape)
print("env.action_space.n = ", env.action_space.n)

obs = env.reset()

for t in range(100):
    obs, reward, is_done, info = env.step(env.action_space.sample())
    print("obs = ", obs)
    print("reward = ", reward)
    print("info = ", info)

    if is_done:
        print("Episode finished after {} timesteps".format(t+1))
        env.reset()
        break

for frame in env.videos:
    print("frame = ", frame)
    video = io.open(frame[0], 'r+b').read()
    encoded = base64.b64encode(video)

    display.display(display.HTML(data="""
        <video alt="test" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        """.format(encoded.decode('ascii'))))

env.observation_space.shape =  (4,)
env.action_space.n =  2
obs =  [ 0.02001072  0.16709003 -0.010497   -0.30946681]
reward =  1.0
info =  {}
obs =  [ 0.02335252  0.36235995 -0.01668633 -0.60544162]
reward =  1.0
info =  {}
obs =  [ 0.03059972  0.16747527 -0.02879516 -0.31806084]
reward =  1.0
info =  {}
obs =  [ 0.03394923 -0.02722498 -0.03515638 -0.03459615]
reward =  1.0
info =  {}
obs =  [ 0.03340473  0.16838303 -0.0358483  -0.3381607 ]
reward =  1.0
info =  {}
obs =  [ 0.03677239 -0.02621096 -0.04261152 -0.0569944 ]
reward =  1.0
info =  {}
obs =  [ 0.03624817  0.16949525 -0.04375141 -0.36281123]
reward =  1.0
info =  {}
obs =  [ 0.03963807  0.36521085 -0.05100763 -0.6689625 ]
reward =  1.0
info =  {}
obs =  [ 0.04694229  0.56100355 -0.06438688 -0.97725933]
reward =  1.0
info =  {}
obs =  [ 0.05816236  0.75692705 -0.08393207 -1.28945229]
reward =  1.0
info =  {}
obs =  [ 0.0733009   0.56296692 -0.10972111 -1.02418333]
reward =  1.0
info =  {}
obs =  [ 0.08456024  0.36946364 -0.130

## Q-learning

Import required packages

In [ ]:
import numpy as np

Brain class

In [ ]:
class Brain:
    def __init__(self, num_states, list_state_range, list_state_reso, num_actions, gamma, r, lr):
        self.num_states = num_states
        self.list_state_range = list_state_range
        self.list_state_reso = list_state_reso
        self.num_actions = num_actions

        self.eps = 1.0  # for epsilon greedy algorithm
        self.gamma = gamma
        self.r = r
        self.lr = lr

        self.q_table = np.random.rand(np.prod(list_state_reso), num_actions)

    def bins(self, clip_min, clip_max, num):
        return np.linspace(clip_min, clip_max, num + 1)[1:-1]
 
    def getStateIndex(self, observation):
        list_index = []
        for i in range(len(self.list_state_reso)):
            index = np.digitize(observation[i], bins=self.bins(self.list_state_range[i][0], self.list_state_range[i][1], self.list_state_reso[i]))
            list_index.append(index)
        return sum([index*int(np.prod(self.list_state_reso[:i])) for i, index in enumerate(list_index)])

    def updateQtable(self, obs, action, reward, next_obs):
        q = self.q_table[self.getStateIndex(obs), action]
        next_q_max = np.max(self.q_table[self.getStateIndex(next_obs)])
        self.q_table[self.getStateIndex(obs), action] = q + self.lr*(reward + self.gamma*next_q_max - q)

    def getAction(self, obs, is_training):
        if is_training and np.random.rand() < self.eps:
            action = np.random.randint(self.num_actions)
        else:
            action = np.argmax(self.q_table[self.getStateIndex(obs)])
        ## update eps
        if is_training and self.eps > 0.1:
            self.eps *= self.r
        return action

Agent class

In [ ]:
class Agent:
    def __init__(self, num_states, list_state_range, list_state_reso, num_actions, gamma, r, lr):
        self.brain = Brain(num_states, list_state_range, list_state_reso, num_actions, gamma, r, lr)
 
    def updateQtable(self, obs, action, reward, next_obs):
        self.brain.updateQtable(obs, action, reward, next_obs)
 
    def getAction(self, obs, is_training):
        action = self.brain.getAction(obs, is_training)
        return action

Environment class

In [ ]:
class Environment:
    def __init__(self, num_episodes, max_step, gamma, r, lr):
        ## parameters
        self.num_episodes = num_episodes
        self.max_step = max_step
        ## environment
        self.env = Monitor(gym.make('CartPole-v0'), './videos/q_learning/', force=True)
        ## agent
        num_states = self.env.observation_space.shape[0]    # position, velocity, angle, angular velocity
        list_state_range = []
        for i in range(num_states):
            list_state_range.append([env.observation_space.low[i], env.observation_space.high[i]])
        list_state_range[1] = [-3.0, 3.0]
        list_state_range[3] = [-0.5, 0.5]
        print("list_state_range = ", list_state_range)
        list_state_reso = [4, 4, 6, 6]
        num_actions = self.env.action_space.n

        self.agent = Agent(num_states, list_state_range, list_state_reso, num_actions, gamma, r, lr)
 
    def train(self):
        num_completed_episodes = 0
  
        for episode in range(self.num_episodes):
            obs = self.env.reset()
            episode_reward = 0
 
            for step in range(self.max_step):
                ## get action
                action = self.agent.getAction(obs, is_training=True)
                ## observe next step
                next_obs, _, is_done, _ = self.env.step(action)
                ## get reward
                if is_done:
                    if step < max_step - 1:
                        reward = -100
                    else:
                        reward = 1
                        num_completed_episodes += 1
                else:
                    reward = 1
                episode_reward += reward
                ## update
                self.agent.updateQtable(obs, action, reward, next_obs)
                ## to next step
                obs = next_obs

                if is_done:
                    print('{0} Episode: Finished after {1} time steps with reward {2}'.format(episode, step+1, episode_reward))
                    break
        print("num_completed_episodes = ", num_completed_episodes)

    def evaluate(self):
        obs = self.env.reset()
        
        for step in range(self.max_step):
            ## get action
            action = self.agent.getAction(obs, is_training=False)
            ## observe next step
            next_obs, _, is_done, _ = self.env.step(action)
            ## to next step
            obs = next_obs

            if is_done:
                print('Evaluation: Finished after {} time steps'.format(step+1))
                break

Prepare showing videos of the restults

In [ ]:
def show_video(env):
    for frame in env.videos:
        print("frame = ", frame)
        video = io.open(frame[0], 'r+b').read()
        encoded = base64.b64encode(video)

        display.display(display.HTML(data="""
            <video alt="test" controls>
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>
            """.format(encoded.decode('ascii')))
        )

Run training

In [ ]:
## parameters
num_episodes = 500
max_step = 200
gamma = 0.9
r = 0.99
lr = 0.5

## run
cartpole_env = Environment(num_episodes, max_step, gamma, r, lr)
cartpole_env.train()
print("Evaluation")
cartpole_env.evaluate()
show_video(cartpole_env.env)

list_state_range =  [[-4.8, 4.8], [-3.0, 3.0], [-0.41887903, 0.41887903], [-0.5, 0.5]]
0 Episode: Finished after 13 time steps with reward -88
1 Episode: Finished after 72 time steps with reward -29
2 Episode: Finished after 10 time steps with reward -91
3 Episode: Finished after 28 time steps with reward -73
4 Episode: Finished after 39 time steps with reward -62
5 Episode: Finished after 32 time steps with reward -69
6 Episode: Finished after 14 time steps with reward -87
7 Episode: Finished after 31 time steps with reward -70
8 Episode: Finished after 9 time steps with reward -92
9 Episode: Finished after 47 time steps with reward -54
10 Episode: Finished after 37 time steps with reward -64
11 Episode: Finished after 58 time steps with reward -43
12 Episode: Finished after 10 time steps with reward -91
13 Episode: Finished after 11 time steps with reward -90
14 Episode: Finished after 42 time steps with reward -59
15 Episode: Finished after 48 time steps with reward -53
16 Episode: 

frame =  ('/content/videos/q_learning/openaigym.video.1.59.video000001.mp4', '/content/videos/q_learning/openaigym.video.1.59.video000001.meta.json')


frame =  ('/content/videos/q_learning/openaigym.video.1.59.video000008.mp4', '/content/videos/q_learning/openaigym.video.1.59.video000008.meta.json')


frame =  ('/content/videos/q_learning/openaigym.video.1.59.video000027.mp4', '/content/videos/q_learning/openaigym.video.1.59.video000027.meta.json')


frame =  ('/content/videos/q_learning/openaigym.video.1.59.video000064.mp4', '/content/videos/q_learning/openaigym.video.1.59.video000064.meta.json')


frame =  ('/content/videos/q_learning/openaigym.video.1.59.video000125.mp4', '/content/videos/q_learning/openaigym.video.1.59.video000125.meta.json')


frame =  ('/content/videos/q_learning/openaigym.video.1.59.video000216.mp4', '/content/videos/q_learning/openaigym.video.1.59.video000216.meta.json')


frame =  ('/content/videos/q_learning/openaigym.video.1.59.video000343.mp4', '/content/videos/q_learning/openaigym.video.1.59.video000343.meta.json')


## References
- [Getting Started with Gym](https://gym.openai.com/docs/)
- [【強化学習】OpenAI Gym を Google Colab上で描画する方法 (2020.6版)](https://qiita.com/ymd_h/items/c393797deb72e1779269)
- [minnano_rl/section_2/01_simple_reinforcement_learning.ipynb](https://github.com/yukinaga/minnano_rl/blob/main/section_2/01_simple_reinforcement_learning.ipynb)
- [第10回　CartPole課題をQ学習で制御する](https://book.mynavi.jp/manatee/detail/id=88997)